In [1]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630184 sha256=b84415a05f755ded32704de93520ced51854e3b8a406369e464d2b3fd70682c8
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


##Install kaggel to load data

In [2]:
! pip install kaggle


In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets download netflix-inc/netflix-prize-data

 99% 677M/683M [00:03<00:00, 244MB/s]
100% 683M/683M [00:03<00:00, 234MB/s]


In [13]:
! unzip /content/netflix-prize-data.zip

Archive:  /content/netflix-prize-data.zip
  inflating: README                  
  inflating: combined_data_1.txt     
  inflating: combined_data_2.txt     
  inflating: combined_data_3.txt     
  inflating: combined_data_4.txt     
  inflating: movie_titles.csv        
  inflating: probe.txt               
  inflating: qualifying.txt          


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

###The first 1,000,000 records from each of the training data files loaded into memory

In [2]:
def readFile(file_path,rows=1000000):
    data_dict = {'Cust_Id' : [], 'Movie_Id' : [], 'Rating' : [], 'Date' : []}
    f = open(file_path, "r")
    count = 0
    for line in f:
        count += 1
        if count > rows:
            break            
        if ':' in line:
            movidId = line[:-2] # remove the last character ':'
            movieId = int(movidId)
        else:
            customerID, rating, date = line.split(',')
            data_dict['Cust_Id'].append(customerID)
            data_dict['Movie_Id'].append(movieId)
            data_dict['Rating'].append(rating)
            data_dict['Date'].append(date.rstrip("\n"))
    f.close()
            
    return pd.DataFrame(data_dict)

In [3]:
df1 = readFile("/content/combined_data_1.txt",rows=1000000)
df2 = readFile("/content/combined_data_2.txt",rows=1000000)
df3 = readFile("/content/combined_data_3.txt",rows=1000000)
df4 = readFile("/content/combined_data_4.txt",rows=1000000)

In [4]:
df1['Rating'] = df1['Rating'].astype(float)
df2['Rating'] = df2['Rating'].astype(float)
df3['Rating'] = df3['Rating'].astype(float)
df4['Rating'] = df4['Rating'].astype(float)

##Combine Datasets

In [5]:
df = df1.copy()
df = df.append(df2)
df = df.append(df3)
df = df.append(df4)
df.index = np.arange(0,len(df))

In [6]:
df.head()

,Cust_Id,Movie_Id,Rating,Date
0,1488844,1,3.0,2005-09-06
1,822109,1,5.0,2005-05-13
2,885013,1,4.0,2005-10-19
3,30878,1,4.0,2005-12-26
4,823519,1,3.0,2004-05-03


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999227 entries, 0 to 3999226
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_Id   object 
 1   Movie_Id  int64  
 2   Rating    float64
 3   Date      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 152.6+ MB


In [8]:
df.describe()

,Movie_Id,Rating
count,3.999227e+06,3.999227e+06
mean,6.873602e+03,3.612618e+00
std,4.994948e+03,1.073674e+00
min,1.000000e+00,1.000000e+00
25%,4.500000e+03,3.000000e+00
50%,9.211000e+03,4.000000e+00
75%,1.336800e+04,4.000000e+00
max,1.351200e+04,5.000000e+00


In [9]:
df_title = pd.read_csv('/content/movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])
df_title.head(10)

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


#Training Model

In [10]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)
svd = SVD()

In [11]:
# Run 5-fold cross-validation and print results
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9615  0.9605  0.9624  0.9611  0.9617  0.9614  0.0006  
MAE (testset)     0.7521  0.7515  0.7531  0.7517  0.7532  0.7523  0.0007  
Fit time          202.71  211.36  207.42  206.01  201.22  205.74  3.58    
Test time         17.31   15.04   15.34   16.76   16.38   16.17   0.85    


{'fit_time': (202.7060992717743,
  211.35864734649658,
  207.41666722297668,
  206.00538516044617,
  201.21695733070374),
 'test_mae': array([0.75208035, 0.75147897, 0.75306165, 0.75173074, 0.75316113]),
 'test_rmse': array([0.96147511, 0.96048369, 0.96241978, 0.96111767, 0.96166735]),
 'test_time': (17.311129093170166,
  15.03968858718872,
  15.342870712280273,
  16.755815505981445,
  16.384320497512817)}

##Re-train the model using the entire training dataset

In [12]:
trainset = data.build_full_trainset()
svd.fit(trainset)

##Predict

In [13]:
titles = df_title.copy()
titles['Estimate_Score'] = titles['Movie_Id'].apply(lambda x: svd.predict(785314, x).est)

In [14]:
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
titles.head(10)

,Movie_Id,Year,Name,Estimate_Score
13503,13504,2004.0,House,4.626936
9435,9436,2002.0,Full Metal Panic FUMOFFU,4.528119
12,13,2003.0,Lord of the Rings: The Return of the King: Ext...,4.378500
9325,9326,2003.0,Queer as Folk: Season 3,4.358383
67,68,2004.0,Invader Zim,4.289657
4632,4633,1982.0,Gandhi,4.278440
4605,4606,2003.0,Scrapped Princess,4.277264
208,209,1996.0,Star Trek: Deep Space Nine: Season 5,4.276553
134,135,1998.0,GTO: Great Teacher Onizuka: Set 2,4.276157
13467,13468,1992.0,Law & Order: Season 3,4.263910
